# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data = pd.read_csv('../output_data/WeatherPy_Data.csv')
data.head(2)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,lardos,GR,36.09,28.02,87.8,52,0,18.34,1595434134
1,road town,VG,18.42,-64.62,87.8,66,40,11.41,1595434134


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
import gmaps
gmaps.configure(api_key=g_key)

In [4]:
locations = data[['Lat', 'Lng']]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = data.Humidity,
                                dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel = data.loc[(data['Max Temp'] > 74) & (data['Max Temp'] < 85)\
                          & (data['Humidity'] < 50) & (data['Cloudiness'] < 20)\
                          & (data['Wind Speed'] < 10), :]
hotel.shape

(9, 9)

###### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel = hotel.copy()
hotel.insert(2, 'Hotel Name', '')
hotel.head(2)

,City,Country,Hotel Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
104,barra,BR,,-11.09,-43.14,84.92,38,0,9.04,1595434152
283,saint george,US,,37.10,-113.58,84.20,25,1,6.93,1595433897


In [7]:
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {'key': g_key, 'radius': 5000, 'type': 'lodging', 'keyword': 'hotel'}

for index, row in hotel.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat}, {lng}'
    response = requests.get(url, params).json()
    try:
        hotel.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except:
        pass


In [8]:
hotel.head(2)

,City,Country,Hotel Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
104,barra,BR,Hotel Do Sertão,-11.09,-43.14,84.92,38,0,9.04,1595434152
283,saint george,US,Best Western Plus Abbey Inn,37.10,-113.58,84.20,25,1,6.93,1595433897


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

# markers = gmaps.marker_layer(marker_locations,
#     info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])

Figure(layout=FigureLayout(height='420px'))